In [ ]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
from sympy.printing.octave import octave_code

mechanics_printing()

class RobotMathEngine6DOF:
    def __init__(self, joint_config, link_vectors_mask):
        self.joint_config = joint_config
        # Converte para Matrix para garantir precisão e evitar floats
        self.link_vectors_mask = [sp.Matrix(v) for v in link_vectors_mask]

        self.t = sp.symbols('t')
        self.g = sp.symbols('g')

        self.q = []
        self.dq = []
        self.params_list = []

        self.frames = []
        self.rotation_matrices = []
        self.com_positions_global = []
        self.angular_velocities = []
        self.masses = []

        self.M = None
        self.G_vec = None
        self.C_total = None # Vetor ÚNICO (Muito mais rápido)
        self.Jacobian = None
        self.M_cse_replacements = []
        self.M_cse = None

    def _rot_matrix_local(self, axis, angle):
        c, s = sp.cos(angle), sp.sin(angle)
        if axis == 'x': return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
        if axis == 'y': return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
        if axis == 'z': return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
        return sp.eye(3)

    def _get_axis_vector(self, axis):
        if axis == 'x': return sp.Matrix([1, 0, 0])
        if axis == 'y': return sp.Matrix([0, 1, 0])
        if axis == 'z': return sp.Matrix([0, 0, 1])
        return sp.Matrix([0,0,0])

    def step_1_kinematics(self):
        print("1. Cinemática (Matrizes de Rotação e CMs)...")
        T_acc = sp.eye(4)
        R_acc = sp.eye(3)
        omega_acc = sp.Matrix([0, 0, 0])

        for i, (j_type, link_vec) in enumerate(zip(self.joint_config, self.link_vectors_mask)):
            q = dynamicsymbols(f'q{i+1}')
            dq = dynamicsymbols(f'q{i+1}', 1)
            self.q.append(q)
            self.dq.append(dq)

            m = sp.symbols(f'm{i+1}')
            L = sp.symbols(f'L{i+1}')
            cx, cy, cz = sp.symbols(f'cx{i+1} cy{i+1} cz{i+1}')
            self.masses.append(m)
            self.params_list.extend([m, L, cx, cy, cz])

            type_char, axis_char = j_type[0], j_type[1].lower()
            axis_vec_local = self._get_axis_vector(axis_char)

            # Rotação
            if type_char == 'R':
                axis_vec_global = R_acc * axis_vec_local
                omega_new = omega_acc + axis_vec_global * dq
                R_j = self._rot_matrix_local(axis_char, q)
                P_j = sp.Matrix([0,0,0])
            elif type_char == 'D':
                omega_new = omega_acc
                R_j = sp.eye(3)
                P_j = sp.Matrix([0,0,0])
                if axis_char == 'x': P_j[0] = q
                if axis_char == 'y': P_j[1] = q
                if axis_char == 'z': P_j[2] = q

            R_acc = R_acc * R_j
            self.rotation_matrices.append(R_acc)
            self.angular_velocities.append(omega_new)
            omega_acc = omega_new

            # Geometria
            T_joint = sp.eye(4)
            T_joint[0:3, 0:3] = R_j
            T_joint[0:3, 3] = P_j
            T_at_joint_start = T_acc * T_joint

            # CM
            v_com_local = sp.Matrix([cx, cy, cz, 1])
            p_com_global = T_at_joint_start * v_com_local
            self.com_positions_global.append(p_com_global[0:3, :])

            # Elo
            P_link_vec = link_vec * L
            T_link = sp.eye(4)
            T_link[0:3, 3] = P_link_vec
            T_acc = T_at_joint_start * T_link
            self.frames.append(T_acc)

    def step_2_jacobian_M_G(self):
        print("2. Dinâmica M e G (Método Jacobianos Rápido)...")
        n = len(self.q)
        self.M = sp.zeros(n, n)
        V_tot = 0

        for i in range(n):
            m = self.masses[i]
            pos_cm = self.com_positions_global[i]
            R_global = self.rotation_matrices[i]

            J_v = pos_cm.jacobian(self.q)
            J_w = self.angular_velocities[i].jacobian(self.dq)

            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            self.params_list.extend([Ixx, Iyy, Izz])
            I_local = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
            I_global = R_global * I_local * R_global.T

            self.M += m * J_v.T * J_v + J_w.T * I_global * J_w
            V_tot += m * self.g * pos_cm[2]

        self.G_vec = sp.Matrix([V_tot]).jacobian(self.q).T
        self.Jacobian = self.frames[-1][0:3, 3].jacobian(self.q)

        print("   Compactando M com CSE para reuso nas derivadas...")
        replacements, reduced = sp.cse(self.M)
        self.M_cse_replacements = replacements
        self.M_cse = reduced[0] if reduced else self.M

    def step_3_coriolis_combined(self):
        print("3. Dinâmica Coriolis Total (Sem separação)...")
        n = len(self.q)
        self.C_total = sp.zeros(n, 1)

        print(f"   Calculando derivadas da Matriz M (Isso pode levar um momento)...")

        # --- CORREÇÃO AQUI ---
        # Não usamos o CSE para derivar. Usamos a matriz M cheia.
        # Isso garante que a regra da cadeia (Chain Rule) funcione corretamente.
        dM_dq = []
        for i, qk in enumerate(self.q):
            # print(f"      Derivando M em relação a q{i+1}...") # Descomente se quiser ver o progresso
            dM_dq.append(self.M.diff(qk))

        print(f"   Montando vetor de Christoffel...")

        for i in range(n):
            termo_linha = 0
            for j in range(n):
                for k in range(j, n): # Otimização de simetria (k começa de j)
                    # Símbolo de Christoffel do Primeiro Tipo
                    # c_ijk = 0.5 * ( dM_ij/dq_k + dM_ik/dq_j - dM_jk/dq_i )

                    dM_ij_dk = dM_dq[k][i, j]
                    dM_ik_dj = dM_dq[j][i, k]
                    dM_jk_di = dM_dq[i][j, k]

                    c_ijk = sp.Rational(1,2) * (dM_ij_dk + dM_ik_dj - dM_jk_di)

                    if c_ijk != 0:
                        # V = c_ijk * dq_j * dq_k
                        termo = c_ijk * self.dq[j] * self.dq[k]

                        # Se k != j, o termo aparece duas vezes na soma (simetria j,k)
                        # pois dq_j*dq_k é igual a dq_k*dq_j
                        if k != j:
                            termo *= 2
                        termo_linha += termo

            # collect ajuda a agrupar termos semelhantes e limpar a expressão
            self.C_total[i] = sp.collect(termo_linha, self.dq)
            print(f"   -> Linha {i+1} ok")

    def step_4_export_cse(self):
        print("\n" + "="*40)
        print(" OTIMIZANDO E EXPORTANDO (CSE) ")
        print("="*40)

        mapa_subs = {}
        for i in range(len(self.q)):
            sym_q = sp.Symbol(f'q{i+1}')
            sym_dq = sp.Symbol(f'dq{i+1}')
            mapa_subs[self.q[i]] = sym_q
            mapa_subs[self.dq[i]] = sym_dq

        # Agrupa expressões
        expressoes = [
            self.M.subs(mapa_subs),
            self.G_vec.subs(mapa_subs),
            self.C_total.subs(mapa_subs),
            self.Jacobian.subs(mapa_subs)
        ]

        print("   Compactando equações (Isso reduz o tamanho do código)...")
        replacements, reduced = sp.cse(expressoes)

        print("% --- Variáveis Auxiliares ---")
        for sym, expr in replacements:
            print(f"{sym} = {octave_code(expr)};")

        M_opt, G_opt, C_opt, J_opt = reduced

        print("\n% --- Matrizes Finais ---")
        print("M = " + octave_code(M_opt) + ";")
        print("G = " + octave_code(G_opt) + ";")
        print("V_total = " + octave_code(C_opt) + "; % Coriolis + Centripeta Juntos")
        print("J = " + octave_code(J_opt) + ";")

# --- RODAR PARA O ANTROPOMÓRFICO ---
antropo_joints = ['Rz', 'Ry', 'Ry', 'Rz', 'Ry', 'Rz']
antropo_links = [[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]

bot_6dof = RobotMathEngine6DOF(antropo_joints, antropo_links)
bot_6dof.step_1_kinematics()
bot_6dof.step_2_jacobian_M_G()
bot_6dof.step_3_coriolis_combined()
bot_6dof.step_4_export_cse()


1. Cinemática (Matrizes de Rotação e CMs)...
2. Dinâmica M e G (Método Jacobianos Rápido)...
   Compactando M com CSE para reuso nas derivadas...
3. Dinâmica Coriolis Total (Sem separação)...
   Calculando derivadas da Matriz M (Isso pode levar um momento)...
   Montando vetor de Christoffel...
   -> Linha 1 ok
   -> Linha 2 ok
   -> Linha 3 ok
   -> Linha 4 ok
   -> Linha 5 ok
   -> Linha 6 ok

 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = sin(q2);
x1 = x0.^2;
x2 = Ixx2.*x1;
x3 = cos(q3);
x4 = x0.*x3;
x5 = sin(q3);
x6 = cos(q2);
x7 = x5.*x6;
x8 = x4 + x7;
x9 = -x8;
x10 = x9.^2;
x11 = x6.^2;
x12 = Izz2.*x11;
x13 = x0.*x5;
x14 = x13 - x3.*x6;
x15 = -x14;
x16 = x15.^2;
x17 = cos(q1);
x18 = cx1.*x17;
x19 = sin(q1);
x20 = -cy1.*x19 + x18;
x21 = cx1.*x19;
x22 = cy1.*x17;
x23 = -x21 - x22;
x24 = cy2.*x17;
x25 = cx2.*x6;
x26 = x19.*x25;
x27 = cz2.*x0;
x28 = x19.*x27;
x29 = x26 + x28;
x30 = -x24 - x29;
x31 = x17.

In [ ]:
# --- RODAR PARA O UVMS ---
# --- 1. CONFIGURAÇÃO DO VEÍCULO (BASE FLUTUANTE) ---
# O veículo possui 6 graus de liberdade livres.
# Modelamos isso como 3 juntas prismáticas (XYZ) e 3 rotativas (RPY).
# Nota: "D" = Prismática (Deslocamento), "R" = Rotativa

vehicle_joints = ['Dx', 'Dy', 'Dz', 'Rz', 'Ry', 'Rx']

# Os "elos" virtuais do veículo geralmente têm comprimento zero,
# pois todos os movimentos ocorrem em torno do centro de flutuabilidade/massa.
vehicle_links = [[0, 0, 0]] * 6

# --- 2. CONFIGURAÇÃO DO BRAÇO (ANTROPOMÓRFICO) ---
# A configuração que você passou:
arm_joints = ['Rz', 'Ry', 'Ry', 'Rz', 'Ry', 'Rz']
arm_links =  [[0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]]

# --- 3. MONTAGEM DO UVMS (12-DOF) ---
uvms_joints_total = vehicle_joints + arm_joints
uvms_links_total  = vehicle_links + arm_links
bot_UVMS = RobotMathEngine6DOF(uvms_joints_total, uvms_links_total)
bot_UVMS.step_1_kinematics()
bot_UVMS.step_2_jacobian_M_G()
bot_UVMS.step_3_coriolis_combined()
bot_UVMS.step_4_export_cse()


A saída de streaming foi truncada nas últimas 5000 linhas.
x5082 = x5081/2;
x5083 = x199.*x4281;
x5084 = x5083/2;
x5085 = x24.*x4268;
x5086 = x5085/2;
x5087 = x24.*x4273;
x5088 = x5087/2;
x5089 = x24.*x4280;
x5090 = x5089/2;
x5091 = x4267.*x829;
x5092 = x5091/2;
x5093 = x4271.*x829;
x5094 = x5093/2;
x5095 = x4278.*x829;
x5096 = x5095/2;
x5097 = x5080 + x5082 + x5084 + x5086 + x5088 + x5090 + x5092 + x5094 + x5096;
x5098 = x4583.*x708;
x5099 = x5098/2;
x5100 = x4589.*x708;
x5101 = x5100/2;
x5102 = x4595.*x708;
x5103 = x5102/2;
x5104 = x396.*x4584;
x5105 = x5104/2;
x5106 = x396.*x4592;
x5107 = x5106/2;
x5108 = x396.*x4596;
x5109 = x5108/2;
x5110 = x4582.*x965;
x5111 = x5110/2;
x5112 = x4587.*x965;
x5113 = x5112/2;
x5114 = x4594.*x965;
x5115 = x5114/2;
x5116 = x1061.*x636;
x5117 = x5116/2;
x5118 = x1065.*x636;
x5119 = x5118/2;
x5120 = x1069.*x636;
x5121 = x5120/2;
x5122 = x1087.*x297;
x5123 = x5122/2;
x5124 = x1090.*x297;
x5125 = x5124/2;
x5126 = x1092.*x297;
x5127 = x5126/2;
x5128 = x103

In [ ]:
import sympy as sp

# --- CONFIGURAÇÃO DA VALIDAÇÃO ---
# Robô 2 Links Planar (Como na imagem do Niku)
inputs_joints = ['Rz', 'Rz']
inputs_links = [[1, 0, 0], [1, 0, 0]]

# Instancia sua Engine
bot = RobotMathEngine6DOF(inputs_joints, inputs_links)

# Ajuste crítico para bater com a imagem do Niku (Robô Vertical):
# A imagem usa cossenos para gravidade, implicando que 0 graus é horizontal e g atua na vertical.
# Vamos forçar a gravidade na direção X negativa para simular isso na validação de G
bot.g = sp.symbols('g')
# Se a sua engine permite definir o vetor gravidade, seria algo como g_vec = [-g, 0, 0]
# Se não, vamos focar na validação de M e V (Coriolis), que independem da gravidade.

bot.step_1_kinematics()
bot.step_2_jacobian_M_G()
bot.step_3_coriolis_combined()
bot.step_4_export_cse()

print("\n" + "="*40)
print("--- VALIDAÇÃO CRUZADA COM NIKU (IMAGEM) ---")
print("="*40)

# 1. Definição das Variáveis Simbólicas do Niku
m1, m2, l1, l2, g = sp.symbols('m1 m2 L1 L2 g')
q1, q2 = bot.q[0], bot.q[1]
dq1, dq2 = bot.dq[0], bot.dq[1]

# Funções trigonométricas compactas (como no livro)
# C1 = cos(q1), C12 = cos(q1+q2)...
C2 = sp.cos(q2)
S2 = sp.sin(q2)
C1 = sp.cos(q1)
C12 = sp.cos(q1 + q2)

# --- 2. MONTANDO AS MATRIZES DA IMAGEM (MANUALMENTE) ---

# A) Matriz de Massa (M) - Niku
M11_n = (sp.Rational(1,3)*m1*l1**2 + m2*l1**2 + sp.Rational(1,3)*m2*l2**2 + m2*l1*l2*C2)
M12_n = (sp.Rational(1,3)*m2*l2**2 + sp.Rational(1,2)*m2*l1*l2*C2)
M21_n = M12_n # Simétrica
M22_n = (sp.Rational(1,3)*m2*l2**2)
M_Niku = sp.Matrix([[M11_n, M12_n], [M21_n, M22_n]])

# B) Matriz de Coriolis e Centrípeta (V) - Niku
# Na imagem, eles estão separados. Vamos escrever igual e somar.

# Termo Centrípeta (multiplica [dq1^2, dq2^2])
Mat_Centripeta = sp.Matrix([
    [0, -sp.Rational(1,2)*m2*l1*l2*S2],
    [sp.Rational(1,2)*m2*l1*l2*S2, 0]
])
Vec_dq_sq = sp.Matrix([dq1**2, dq2**2])
V_cent_Niku = Mat_Centripeta * Vec_dq_sq

# Termo Coriolis (multiplica [dq1*dq2, dq2*dq1])
Mat_Coriolis = sp.Matrix([
    [-(m2*l1*l2*S2), 0],
    [0, 0]
])
Vec_dq_cross = sp.Matrix([dq1*dq2, dq2*dq1])
V_cor_Niku = Mat_Coriolis * Vec_dq_cross

# VETOR TOTAL DO NIKU (Soma dos dois)
V_total_Niku = V_cent_Niku + V_cor_Niku

# C) Vetor Gravidade (G) - Niku
# Imagem: [(0.5*m1+m2)g*l1*C1 + 0.5*m2*g*l2*C12; 0.5*m2*g*l2*C12]
G1_n = (sp.Rational(1,2)*m1 + m2)*g*l1*C1 + sp.Rational(1,2)*m2*g*l2*C12
G2_n = sp.Rational(1,2)*m2*g*l2*C12
G_Niku = sp.Matrix([G1_n, G2_n])


# --- 3. MAPA DE SUBSTITUIÇÃO (Seu Robô -> Livro) ---
# A sua engine usa parâmetros genéricos (cx, Izz). O livro usa hastes finas.
subs_map = {
    # Geometria
    sp.symbols('L1'): l1, sp.symbols('L2'): l2,
    sp.symbols('m1'): m1, sp.symbols('m2'): m2,

    # Centro de Massa (No meio da haste, como assumido pelo Niku 1/3 e 1/12)
    sp.symbols('cx1'): l1/2, sp.symbols('cx2'): l2/2,
    sp.symbols('cy1'): 0, sp.symbols('cz1'): 0,
    sp.symbols('cy2'): 0, sp.symbols('cz2'): 0,

    # Inércias (Haste fina girando pela ponta = 1/3 mL^2, pelo centro = 1/12 mL^2)
    # Sua engine usa Steiner a partir do CM, então usamos I_cm = 1/12 mL^2
    sp.symbols('Izz1'): sp.Rational(1,12)*m1*l1**2,
    sp.symbols('Izz2'): sp.Rational(1,12)*m2*l2**2,

    # Zerar inércias irrelevantes para planar
    sp.symbols('Ixx1'): 0, sp.symbols('Iyy1'): 0,
    sp.symbols('Ixx2'): 0, sp.symbols('Iyy2'): 0
}


# --- 4. COMPARAÇÃO ---

print("\n>>> Comparando MATRIZ DE MASSA (M)...")
Diff_M = sp.simplify(sp.expand(bot.M.subs(subs_map) - M_Niku))
sp.pprint(Diff_M)
if Diff_M == sp.zeros(2,2): print("✅ M: Bateu Perfeito!")
else: print("❌ M: Tem diferença.")

print("\n>>> Comparando CORIOLIS + CENTRÍPETA (V_total)...")
# Nota: bot.C_vec ou bot.C_total dependendo da versão da sua classe
# Assumindo que sua classe salva em bot.C_total ou bot.C_vec
try:
    V_Engine = bot.C_total # Tente C_total (versão otimizada)
except:
    V_Engine = bot.C_vec   # Ou C_vec (versão Christoffel)

Diff_V = sp.simplify(sp.expand(V_Engine.subs(subs_map) - V_total_Niku))
sp.pprint(Diff_V)
if Diff_V == sp.zeros(2,1): print("✅ V (Coriolis+Cent): Bateu Perfeito!")
else: print("❌ V: Tem diferença.")

print("\n>>> Comparando GRAVIDADE (G)...")
# OBS: Isso só vai bater se a sua engine tiver sido configurada
# para gravidade alinhada com o modelo da imagem (vertical).
# Se der erro aqui, é provavelmente apenas a direção do vetor 'g' na sua engine.
G_Engine = bot.G_vec.subs(subs_map)

# Tenta validar G normalizando a direção (ignorando sinal ou eixo se necessário)
# Mas vamos testar direto primeiro
Diff_G = sp.simplify(sp.expand(G_Engine - G_Niku))
# sp.pprint(Diff_G)

if Diff_G == sp.zeros(2,1):
    print("✅ G: Bateu Perfeito!")
else:
    print("⚠️ G: Deu diferença (Esperado se a gravidade da Engine for Z e do Livro for X/Y)")
    print("   Se M e V bateram, sua dinâmica está correta.")

1. Cinemática (Matrizes de Rotação e CMs)...
2. Dinâmica M e G (Método Jacobianos Rápido)...
   Compactando M com CSE para reuso nas derivadas...
3. Dinâmica Coriolis Total (Sem separação)...
   Calculando derivadas da Matriz M (Isso pode levar um momento)...
   Montando vetor de Christoffel...
   -> Linha 1 ok
   -> Linha 2 ok

 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = cos(q1);
x1 = cx1.*x0;
x2 = sin(q1);
x3 = -cy1.*x2 + x1;
x4 = cx1.*x2;
x5 = cy1.*x0;
x6 = -x4 - x5;
x7 = L1.*x0;
x8 = sin(q2);
x9 = cos(q2);
x10 = -x0.*x9 + x2.*x8;
x11 = -x10;
x12 = x0.*x8 + x2.*x9;
x13 = -x12;
x14 = cx2.*x11 + cy2.*x13;
x15 = x14 + x7;
x16 = L1.*x2;
x17 = cy2.*x10;
x18 = cx2.*x13;
x19 = x17 + x18;
x20 = -x16 + x19;
x21 = m2.*x14;
x22 = m2.*x19;
x23 = Izz2 + x15.*x21 + x20.*x22;
x24 = cy2.*x12;
x25 = cx2.*x10;
x26 = 2*x24 + 2*x25;
x27 = m2.*x20;
x28 = x27/2;
x29 = x26.*x28;
x30 = 2*x17 + 2*x18;
x31 = m2.*x15/2;
x32 = 

In [4]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
from sympy.printing.octave import octave_code

mechanics_printing()

class RobotMath:
    def __init__(self, joint_config, link_vectors_mask):
        self.joint_config = joint_config
        # Converte para Matrix para garantir precisão e evitar floats
        self.link_vectors_mask = [sp.Matrix(v) for v in link_vectors_mask]

        self.t = sp.symbols('t')
        self.g = sp.symbols('g')

        self.q = []
        self.dq = []
        self.params_list = []

        self.frames = []
        self.rotation_matrices = []
        self.com_positions_global = []
        self.angular_velocities = []
        self.masses = []

        self.M = None
        self.G_vec = None
        self.C_total = None # Vetor ÚNICO (Muito mais rápido)
        self.Jacobian = None
        self.M_cse_replacements = []
        self.M_cse = None

    def _rot_matrix_local(self, axis, angle):
        c, s = sp.cos(angle), sp.sin(angle)
        if axis == 'x': return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
        if axis == 'y': return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
        if axis == 'z': return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
        return sp.eye(3)

    def _get_axis_vector(self, axis):
        if axis == 'x': return sp.Matrix([1, 0, 0])
        if axis == 'y': return sp.Matrix([0, 1, 0])
        if axis == 'z': return sp.Matrix([0, 0, 1])
        return sp.Matrix([0,0,0])

    def step_1_kinematics(self):
        print("1. Cinemática (Matrizes de Rotação e CMs)...")
        T_acc = sp.eye(4)
        R_acc = sp.eye(3)
        omega_acc = sp.Matrix([0, 0, 0])

        for i, (j_type, link_vec) in enumerate(zip(self.joint_config, self.link_vectors_mask)):
            q = dynamicsymbols(f'q{i+1}')
            dq = dynamicsymbols(f'q{i+1}', 1)
            self.q.append(q)
            self.dq.append(dq)

            m = sp.symbols(f'm{i+1}')
            L = sp.symbols(f'L{i+1}')
            cx, cy, cz = sp.symbols(f'cx{i+1} cy{i+1} cz{i+1}')
            self.masses.append(m)
            self.params_list.extend([m, L, cx, cy, cz])

            type_char, axis_char = j_type[0], j_type[1].lower()
            axis_vec_local = self._get_axis_vector(axis_char)

            # Rotação
            if type_char == 'R':
                axis_vec_global = R_acc * axis_vec_local
                omega_new = omega_acc + axis_vec_global * dq
                R_j = self._rot_matrix_local(axis_char, q)
                P_j = sp.Matrix([0,0,0])
            elif type_char == 'D':
                omega_new = omega_acc
                R_j = sp.eye(3)
                P_j = sp.Matrix([0,0,0])
                if axis_char == 'x': P_j[0] = q
                if axis_char == 'y': P_j[1] = q
                if axis_char == 'z': P_j[2] = q

            R_acc = R_acc * R_j
            self.rotation_matrices.append(R_acc)
            self.angular_velocities.append(omega_new)
            omega_acc = omega_new

            # Geometria
            T_joint = sp.eye(4)
            T_joint[0:3, 0:3] = R_j
            T_joint[0:3, 3] = P_j
            T_at_joint_start = T_acc * T_joint

            # CM
            v_com_local = sp.Matrix([cx, cy, cz, 1])
            p_com_global = T_at_joint_start * v_com_local
            self.com_positions_global.append(p_com_global[0:3, :])

            # Elo
            P_link_vec = link_vec * L
            T_link = sp.eye(4)
            T_link[0:3, 3] = P_link_vec
            T_acc = T_at_joint_start * T_link
            self.frames.append(T_acc)

    def step_2_jacobian_M_G(self):
        print("2. Dinâmica M e G (Método Jacobianos Rápido)...")
        n = len(self.q)
        self.M = sp.zeros(n, n)
        V_tot = 0

        for i in range(n):
            m = self.masses[i]
            pos_cm = self.com_positions_global[i]
            R_global = self.rotation_matrices[i]

            J_v = pos_cm.jacobian(self.q)
            J_w = self.angular_velocities[i].jacobian(self.dq)

            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            self.params_list.extend([Ixx, Iyy, Izz])
            I_local = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
            I_global = R_global * I_local * R_global.T

            self.M += m * J_v.T * J_v + J_w.T * I_global * J_w
            V_tot += m * self.g * pos_cm[2]

        self.G_vec = sp.Matrix([V_tot]).jacobian(self.q).T
        self.Jacobian = self.frames[-1][0:3, 3].jacobian(self.q)

        print("   Compactando M com CSE para reuso nas derivadas...")
        replacements, reduced = sp.cse(self.M)
        self.M_cse_replacements = replacements
        self.M_cse = reduced[0] if reduced else self.M

    def step_3_coriolis_combined(self):
        print("3. Dinâmica Coriolis Total (Sem separação)...")
        n = len(self.q)
        self.C_total = sp.zeros(n, 1)

        print(f"   Calculando derivadas da Matriz M (Isso pode levar um momento)...")

        # --- CORREÇÃO AQUI ---
        # Não usamos o CSE para derivar. Usamos a matriz M cheia.
        # Isso garante que a regra da cadeia (Chain Rule) funcione corretamente.
        dM_dq = []
        for i, qk in enumerate(self.q):
            # print(f"      Derivando M em relação a q{i+1}...") # Descomente se quiser ver o progresso
            dM_dq.append(self.M.diff(qk))

        print(f"   Montando vetor de Christoffel...")

        for i in range(n):
            termo_linha = 0
            for j in range(n):
                for k in range(j, n): # Otimização de simetria (k começa de j)
                    # Símbolo de Christoffel do Primeiro Tipo
                    # c_ijk = 0.5 * ( dM_ij/dq_k + dM_ik/dq_j - dM_jk/dq_i )

                    dM_ij_dk = dM_dq[k][i, j]
                    dM_ik_dj = dM_dq[j][i, k]
                    dM_jk_di = dM_dq[i][j, k]

                    c_ijk = sp.Rational(1,2) * (dM_ij_dk + dM_ik_dj - dM_jk_di)

                    if c_ijk != 0:
                        # V = c_ijk * dq_j * dq_k
                        termo = c_ijk * self.dq[j] * self.dq[k]

                        # Se k != j, o termo aparece duas vezes na soma (simetria j,k)
                        # pois dq_j*dq_k é igual a dq_k*dq_j
                        if k != j:
                            termo *= 2
                        termo_linha += termo

            # collect ajuda a agrupar termos semelhantes e limpar a expressão
            self.C_total[i] = sp.collect(termo_linha, self.dq)
            print(f"   -> Linha {i+1} ok")

    def step_4_export_cse(self):
        print("\n" + "="*40)
        print(" OTIMIZANDO E EXPORTANDO (CSE) ")
        print("="*40)

        mapa_subs = {}
        for i in range(len(self.q)):
            sym_q = sp.Symbol(f'q{i+1}')
            sym_dq = sp.Symbol(f'dq{i+1}')
            mapa_subs[self.q[i]] = sym_q
            mapa_subs[self.dq[i]] = sym_dq

        # Agrupa expressões
        expressoes = [
            self.M.subs(mapa_subs),
            self.G_vec.subs(mapa_subs),
            self.C_total.subs(mapa_subs),
            self.Jacobian.subs(mapa_subs)
        ]

        print("   Compactando equações (Isso reduz o tamanho do código)...")
        replacements, reduced = sp.cse(expressoes)

        print("% --- Variáveis Auxiliares ---")
        for sym, expr in replacements:
            print(f"{sym} = {octave_code(expr)};")

        M_opt, G_opt, C_opt, J_opt = reduced

        print("\n% --- Matrizes Finais ---")
        print("M = " + octave_code(M_opt) + ";")
        print("G = " + octave_code(G_opt) + ";")
        print("V_total = " + octave_code(C_opt) + "; % Coriolis + Centripeta Juntos")
        print("J = " + octave_code(J_opt) + ";")

In [5]:
import gradio as gr
import sympy as sp
from sympy.printing.octave import octave_code
import os

# ==============================================================================
# 1. FUNÇÃO DE PROCESSAMENTO
# ==============================================================================
def processar_e_gerar_arquivos(*args):
    # args[0] = número de juntas ativas
    # args[1:] = lista plana inputs [dropdown, cb_x, cb_y, cb_z, ...]

    num_juntas_ativas = int(args[0])
    dados_juntas = args[1:]

    lista_tipos = []
    lista_elos = []

    # Reconstrói as listas (Dropdown + 3 Checkboxes)
    for i in range(num_juntas_ativas):
        idx_base = i * 4
        tipo = dados_juntas[idx_base]
        x = 1 if dados_juntas[idx_base + 1] else 0
        y = 1 if dados_juntas[idx_base + 2] else 0
        z = 1 if dados_juntas[idx_base + 3] else 0
        lista_tipos.append(tipo)
        lista_elos.append([x, y, z])

    # Instancia a Engine
    try:
        # Tenta usar a Híbrida se disponível (mais rápida)
        if 'RobotMathEngineHybrid' in globals():
            bot = RobotMathEngineHybrid(lista_tipos, lista_elos)
            bot.step_1_kinematics()
            bot.step_2_jacobian_M_G()

            # Tenta calcular Coriolis (pode demorar no 12-DOF)
            try:
                if hasattr(bot, 'step_3_coriolis_combined'):
                    bot.step_3_coriolis_combined()
            except:
                pass
        else:
            # Usa a classe padrão (RobotMath)
            bot = RobotMath(lista_tipos, lista_elos)
            bot.step_1_kinematics()
            bot.step_2_jacobian_M_G()
            bot.step_3_coriolis_combined()

    except Exception as e:
        # Retorna Nones + msg erro (agora são 7 arquivos de saída)
        return [None]*7 + [f"Erro Crítico na Engine: {str(e)}"]

    # --- PREPARAÇÃO PARA EXPORTAÇÃO ---

    # Mapeamento de símbolos (q1, q2, dq1, dq2...)
    mapa_subs = {}
    for i in range(len(bot.q)):
        mapa_subs[bot.q[i]] = sp.Symbol(f'q{i+1}')
        mapa_subs[bot.dq[i]] = sp.Symbol(f'dq{i+1}')

    # Função auxiliar de salvamento
    def salvar(nome, expr):
        caminho = f"{nome}.txt"
        with open(caminho, "w") as f:
            if expr is not None:
                try:
                    # Converte SymPy -> MATLAB
                    if hasattr(expr, 'subs'):
                         txt = octave_code(expr.subs(mapa_subs))
                    else:
                         txt = str(expr)

                    f.write(f"% --- {nome} ---\n")
                    f.write(txt)
                except Exception as e:
                    f.write(f"% Erro na conversão: {str(e)}")
            else:
                f.write("% Não calculado (Null)")
        return caminho

    # --- CÁLCULOS EXTRAS ---

    # 1. Cinemática de Velocidade (V = J * dq)
    # Gera o vetor simbólico de velocidades cartesianas [vx vy vz wx wy wz]'
    if bot.Jacobian is not None:
        vetor_dq = sp.Matrix(bot.dq)
        V_cartesian = bot.Jacobian * vetor_dq
    else:
        V_cartesian = None

    # --- GERAÇÃO DOS ARQUIVOS ---

    # Dinâmica
    path_m = salvar("Matriz_M", bot.M)
    path_g = salvar("Vetor_G", bot.G_vec)
    path_c = salvar("Vetor_C_Coriolis", bot.C_total)

    # Cinemática (Separada)
    # Posição: Matriz T do último elo (contém Rotação e Posição XYZ)
    path_fk_pos = salvar("Cinematica_Posicao_T", bot.frames[-1])
    # Velocidade: Vetor V (contém vel linear e angular explicitas)
    path_fk_vel = salvar("Cinematica_Velocidade_V", V_cartesian)

    # Jacobianos (Úteis para controle)
    path_jl = None
    path_ja = None
    if bot.Jacobian is not None:
        if bot.Jacobian.shape[0] > 3:
            path_jl = salvar("Jacobiano_Linear", bot.Jacobian[0:3, :])
            path_ja = salvar("Jacobiano_Angular", bot.Jacobian[3:6, :])
        else:
            path_jl = salvar("Jacobiano_Linear", bot.Jacobian)

    return path_m, path_g, path_c, path_fk_pos, path_fk_vel, path_jl, path_ja, "Sucesso! Arquivos Gerados."


# ==============================================================================
# 2. INTERFACE GRÁFICA
# ==============================================================================
with gr.Blocks(title="Gerador Robótica TCC") as demo:
    gr.Markdown("## 🤖 Gerador de Modelagem Completo")
    gr.Markdown("Gera Dinâmica (M, C, G) e Cinemática (Posição e Velocidade) separadas.")

    num_juntas = gr.State(value=1)
    lista_rows = []
    lista_inputs = []

    with gr.Column():
        for i in range(12):
            visivel = True if i == 0 else False
            with gr.Row(visible=visivel) as r:
                dd = gr.Dropdown(["Rz", "Ry", "Rx", "Dz", "Dy", "Dx"], value="Rz", label=f"Junta {i+1}", scale=2)
                with gr.Row(scale=3):
                    gr.Markdown(f"**Elo {i+1}:**")
                    cx = gr.Checkbox(label="X", value=False)
                    cy = gr.Checkbox(label="Y", value=False)
                    cz = gr.Checkbox(label="Z", value=(True if i==0 else False))
                lista_inputs.extend([dd, cx, cy, cz])
            lista_rows.append(r)

        btn_add = gr.Button("+ Adicionar Junta")

    btn_calc = gr.Button("Calcular Tudo", variant="primary")

    gr.Markdown("### 📂 Downloads")

    with gr.Row():
        gr.Markdown("**Dinâmica:**")
        file_m = gr.File(label="Matriz M (Inércia)")
        file_g = gr.File(label="Vetor G (Gravidade)")
        file_c = gr.File(label="Vetor C (Coriolis)")

    with gr.Row():
        gr.Markdown("**Cinemática:**")
        file_fk_pos = gr.File(label="Posição (Matriz T)")
        file_fk_vel = gr.File(label="Velocidade (Vetor V)")

    with gr.Row():
        gr.Markdown("**Jacobianos:**")
        file_jl = gr.File(label="Jacobiano Linear")
        file_ja = gr.File(label="Jacobiano Angular")

    lbl_status = gr.Label(value="Aguardando...", label="Status")

    # Lógica
    def mostrar_mais_uma(atual):
        novo = atual + 1 if atual < 12 else 12
        updates = []
        for i in range(12): updates.append(gr.Row(visible=(i < novo)))
        return [novo] + updates

    btn_add.click(fn=mostrar_mais_uma, inputs=[num_juntas], outputs=[num_juntas] + lista_rows)

    btn_calc.click(
        fn=processar_e_gerar_arquivos,
        inputs=[num_juntas] + lista_inputs,
        outputs=[file_m, file_g, file_c, file_fk_pos, file_fk_vel, file_jl, file_ja, lbl_status]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a2506caef299f2b4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
